In [ ]:
!pip install transformers==4.17.0 -q

# Deceptive Opinion Detection using BERT without Fine-Tune
In this notebook, we are going to use pre-trained BERT to process text to learn features. Then, the encoded embeddings are used to train a logistic regression model for classifcation. The used corpus consists of truthful and deceptive hotel reviews of 20 Chicago hotels. The model will be used to classify the review as truthful or decpetive.  The data is open in [Kaggle](https://www.kaggle.com/rtatman/deceptive-opinion-spam-corpus).  

## Agenda

1. Data Loading
2. BaseLine Mode: BoW Features + Logistic Regression
3. BERT without Fine-Tune:

   3.1 DistillBERT is used to process sentences to learn features

   3.2 Features are fed into Logistic Regression for classifcation.

DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.


In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import seaborn as sns
import numpy as np

## 1. Data Loading

In [2]:
basefn = "../data/"
df_corpus = pd.read_csv(basefn + "deceptive_opinion.csv")
df_corpus['LABEL'] = 1
df_corpus.loc[df_corpus['deceptive']=='truthful', 'LABEL'] = 0

In [3]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    # 2. Only keep letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    return( " ".join(words)) 
# Get the number of reviews based on the dataframe column size
num_reviews = df_corpus["text"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( df_corpus["text"][i] ) )

/Users/rz/miniforge3/envs/bt5153env/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [4]:
df_corpus['TEXT'] = clean_train_reviews
df_corpus = df_corpus[['TEXT', 'LABEL']]
df_corpus.head()

,TEXT,LABEL
0,we stayed for a one night getaway with family ...,0
1,triple a rate with upgrade to view room was le...,0
2,this comes a little late as i m finally catchi...,0
3,the omni chicago really delivers on all fronts...,0
4,i asked for a high floor away from the elevato...,0


## 2. Baseline Model

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
df_train, df_test, train_labels, test_labels = train_test_split(df_corpus.TEXT, df_corpus.LABEL, test_size=0.25, random_state=123)

#### BoW Model

In [7]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(df_train)
X_test =vectorizer.transform(df_test)

#### Logistic Regression

In [8]:
lr_clf = LogisticRegression(max_iter=20000)
lr_clf.fit(X_train, train_labels)

LogisticRegression(max_iter=20000)

#### Evaluating the baseline model

In [9]:
lr_clf.score(X_test, test_labels)

0.8225

## 3. BERT without Fine-Tune

#### Load pre-trained model

In [10]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# Load pre-trained model (weights)
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

/Users/rz/miniforge3/envs/bt5153env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|████████████████████████████| 226k/226k [00:00<00:00, 232kB/s]
Downloading: 100%|███████████████████████████| 28.0/28.0 [00:00<00:00, 6.04kB/s]
Downloading: 100%|██████████████████████████████| 483/483 [00:00<00:00, 179kB/s]
Downloading: 100%|███████████████████████████| 256M/256M [00:16<00:00, 15.9MB/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architectur

#### 3.1 Convert text into vectors using DistillBERT

### Tokenization-Padding-Masking

In [11]:
tokenized = df_corpus['TEXT'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1600, 512)

In [12]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [13]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1600, 512)

### Batch Inference

To save memory, 10 reviews are fed into the BERT model each time.

In [14]:
feature_list = []
with torch.no_grad():
    for batch_idx in range(0,padded.shape[0],10):
        #BERT check 10 sample each time.
        input_ids = torch.tensor(padded[batch_idx:batch_idx+10])  
        used_attention_mask = torch.tensor(attention_mask[batch_idx:batch_idx+10])
        last_hidden_states = model(input_ids, attention_mask=used_attention_mask)
        #Get the embeddings for the [CLS] tag (position is 0)
        features = last_hidden_states[0][:,0,:].numpy()
        feature_list.append(features)

It should be noted that although the `[CLS]` acts as an "aggregate representation" for classification tasks, this is not the best choice for a high quality sentence embedding vector. [According to](https://github.com/google-research/bert/issues/164) BERT author Jacob Devlin: "*I'm not sure what these vectors are, since BERT does not generate meaningful sentence vectors. It seems that this is is doing average pooling over the word tokens to get a sentence vector, but we never suggested that this will generate meaningful sentence representations*."

(However, the [CLS] token does become meaningful if the model has been fine-tuned, where the last hidden layer of this token is used as the "sentence vector" for sequence classification.)

![picture](https://github.com/rz0718/colab_imgs/blob/main/imgs/bert_output_sentence.png?raw=true)

In [15]:
# preprare features
features = np.vstack(feature_list)
features.shape

(1600, 768)

#### 3.2 Build Logistic Regression

In [16]:
#get labels
labels = df_corpus.LABEL.tolist()

In [17]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=123)

In [18]:
lr_clf = LogisticRegression(max_iter=20000)
lr_clf.fit(train_features, train_labels)

LogisticRegression(max_iter=20000)

In [19]:
#validate the model
lr_clf.score(test_features, test_labels)

0.845

So it is clear that the features encoded by pre-trained BERT is better than the BoW features.  
And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.